In [1]:
import alignments as alg
import utils

In [2]:
human = utils.read_protein('data/alg_HumanEyelessProtein.txt')
fly = utils.read_protein('data/alg_FruitflyEyelessProtein.txt')
#print(len(human))
#print(len(fly))
scoring_matrix = utils.read_scoring_matrix('data/alg_PAM50.txt')
alignment_matrix = alg.compute_alignment_matrix(human, fly, scoring_matrix, False)

score, align_human, align_fly = alg.compute_local_alignment(human, fly, scoring_matrix, alignment_matrix)
print("Local alignment score: " + str(score))
print("Local alignment of human protein: " + align_human)
print("Local alignment of fly protein: " + align_fly)

Local alignment score: 875
Local alignment of human protein: HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSINRVLRNLASEK-QQ
Local alignment of fly protein: HSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATAEVVSKISQYKRECPSIFAWEIRDRLLQENVCTNDNIPSVSSINRVLRNLAAQKEQQ
